### Create a SQL DB from a series of processed CSV files via SQLAlchemy.

The processed CSV files were imported from the WHO, UN, OECD, World Bank, loaded, processed and exported via Pandas


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Create a list (array) with the names of the CSV files from the processed data location

In [2]:
datapath = os.path.join(os.getcwd(), "data2")

if os.path.isdir(datapath):
    datafiles = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath, f))]

For these files, split the extension, capitalize and append them to a dictionary which will contain as key the name minus extension and as value the Pandas DataFrame

In [3]:
datadict = dict()

for f in datafiles:
    #print(f)
    datadict[((f.rsplit(".", 1)[0]).capitalize())] = pd.read_csv(os.path.join(datapath, f))#, index_col = "Country")

In [4]:
for key, val in datadict.items():
    print(key)

Anual_number_of_deaths_by_cause
Life_satisfaction_in_cantril_ladder_world_happiness_report_2021
Life_expectancy
Life_expectancy_at_birth
Extreme_poverty_headcount_ratio_vs_life_expectancy_at_birth
Human_development_index
Mortality_rate_under_5_per_1000_live_births
Suicide_mortality_rate_per_100000_population
Annual_co2_emissions
Average_total_years_of_schooling_for_adult_population


In [37]:
# find for each DB, the unique country/index entries
# which we convert to lists and then sets to find the
# intersection of the lists/sets

uniqndx = []

for key, val in datadict.items():
    uniqndx.append(list(datadict[key]["Country"].unique()))

In [38]:
#inter = [i for i in uniqndx[0]]
countrylist = []

for i, val in enumerate(uniqndx):
    countrylist = list(set(countrylist).intersection(set(uniqndx[i]))) \
        if i != 0 else [i for i in uniqndx[0]]

In [7]:
for key, val in datadict.items():
    print(key)

Anual_number_of_deaths_by_cause
Life_satisfaction_in_cantril_ladder_world_happiness_report_2021
Life_expectancy
Life_expectancy_at_birth
Extreme_poverty_headcount_ratio_vs_life_expectancy_at_birth
Human_development_index
Mortality_rate_under_5_per_1000_live_births
Suicide_mortality_rate_per_100000_population
Annual_co2_emissions
Average_total_years_of_schooling_for_adult_population


In [9]:
df = pd.DataFrame()

### Check for NaNs

we're going to have NaNs since multiple dataframes span different time ranges. Since we do an union of the dataframes we're guaranteed to have some fields/columns without data depending on the year.
If there are any *Code* fields left with NaN, fill to *NUL*. For everything else fill them with 0.


In [10]:
# NEEDED for geo maps

for key, val in datadict.items():
    print(key)
    datadict[key].drop(columns=["Code"], inplace = True)

Anual_number_of_deaths_by_cause
Life_satisfaction_in_cantril_ladder_world_happiness_report_2021
Life_expectancy
Life_expectancy_at_birth
Extreme_poverty_headcount_ratio_vs_life_expectancy_at_birth
Human_development_index
Mortality_rate_under_5_per_1000_live_births
Suicide_mortality_rate_per_100000_population
Annual_co2_emissions
Average_total_years_of_schooling_for_adult_population


In [11]:
df = pd.merge(left=datadict["Anual_number_of_deaths_by_cause"], right=datadict["Average_total_years_of_schooling_for_adult_population"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Life_satisfaction_in_cantril_ladder_world_happiness_report_2021"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Life_expectancy"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Life_expectancy_at_birth"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Extreme_poverty_headcount_ratio_vs_life_expectancy_at_birth"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Human_development_index"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Mortality_rate_under_5_per_1000_live_births"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Suicide_mortality_rate_per_100000_population"], how="outer", on=["Year", "Country"])

df = pd.merge(left=df, right=datadict["Annual_co2_emissions"], how="outer", on=["Year", "Country"])

df.head()

,Country,Year,Number_of_executions_(amnesty_international),Deaths_-_meningitis,Deaths_-_lower_respiratory_infections,Deaths_-_intestinal_infectious_diseases,Deaths_-_protein-energy_malnutrition,Terrorism,Deaths_-_cardiovascular_diseases,Deaths_-_alzheimer_disease_and_other_dementias,...,Life_satisfaction_in_Cantril_Ladder_(World_Happiness_Report_2021),Life_expectancy,Life_expectancy_at_birth,$1.90_per_day_-_share_of_population_below_poverty_line,"Life_expectancy_at_birth,_total_(years)","Total_population_(Gapminder,_HYDE_&_UN)",Human_development_index,Mortality_rate_under_5_per_1000_live_births,Suicidy_mortality_rate_per_100000_population,Annual_CO2_emissions
0,Afghanistan,1990,0,6469.977091,22836.912346,295.382207,1607.703797,12.0,46498.085024,1959.215373,...,NaN,50.331,50.331,0.0,50.331,12412311.0,NaN,17.77,NaN,2603238.0
1,Afghanistan,1991,0,6347.158764,22325.633931,303.866598,1558.132316,68.0,46967.361037,1987.812878,...,NaN,50.999,50.999,0.0,50.999,13299016.0,NaN,17.17,NaN,2427366.0
2,Afghanistan,1992,0,6659.741428,23205.280750,317.750121,1617.721576,49.0,48355.558515,2025.102326,...,NaN,51.641,51.641,0.0,51.641,14485543.0,NaN,16.59,NaN,1379462.0
3,Afghanistan,1993,0,8068.386263,28229.715397,333.978331,1931.808553,0.0,50072.563899,2064.827903,...,NaN,52.256,52.256,0.0,52.256,15816601.0,NaN,16.04,NaN,1333055.0
4,Afghanistan,1994,0,9432.845127,32652.297630,348.419085,2351.629206,22.0,51416.815442,2101.020630,...,NaN,52.842,52.842,0.0,52.842,17075728.0,NaN,15.52,NaN,1281602.0


In [12]:
df.columns

Index(['Country', 'Year', 'Number_of_executions_(amnesty_international)',
       'Deaths_-_meningitis', 'Deaths_-_lower_respiratory_infections',
       'Deaths_-_intestinal_infectious_diseases',
       'Deaths_-_protein-energy_malnutrition', 'Terrorism',
       'Deaths_-_cardiovascular_diseases',
       'Deaths_-_alzheimer_disease_and_other_dementias',
       'Deaths_-_chronic_kidney_disease',
       'Deaths_-_chronic_respiratory_diseases',
       'Deaths_-_cirrhosis_and_other_chronic_liver_diseases',
       'Deaths_-_digestive_diseases', 'Deaths_-_hepatitis',
       'Deaths_-_neoplasms', 'Deaths_-_parkinson_disease',
       'Deaths_-_fire,_heat,_and_hot_substances', 'Deaths_-_malaria',
       'Deaths_-_drowning', 'Deaths_-_interpersonal_violence',
       'Deaths_-_hiv/aids', 'Deaths_-_drug_use_disorders',
       'Deaths_-_tuberculosis', 'Deaths_-_road_injuries',
       'Deaths_-_maternal_disorders', 'Deaths_-_neonatal_disorders',
       'Deaths_-_alcohol_use_disorders',
       'Deaths

In [13]:
df.rename(columns={"Number_of_executions_(amnesty_international)" : "Number_of_executions", "Life_satisfaction_in_Cantril_Ladder_(World_Happiness_Report_2021)" : "Life_satisfaction", "$1.90_per_day_-_share_of_population_below_poverty_line" : "Share_of_population_below_poverty_line_2USD_per_day", "Total_population_(Gapminder,_HYDE_&_UN)" : "Total_population"}, inplace=True)

In [14]:
df.drop(columns={"Number_of_executions"}, inplace=True)
df.columns

Index(['Country', 'Year', 'Deaths_-_meningitis',
       'Deaths_-_lower_respiratory_infections',
       'Deaths_-_intestinal_infectious_diseases',
       'Deaths_-_protein-energy_malnutrition', 'Terrorism',
       'Deaths_-_cardiovascular_diseases',
       'Deaths_-_alzheimer_disease_and_other_dementias',
       'Deaths_-_chronic_kidney_disease',
       'Deaths_-_chronic_respiratory_diseases',
       'Deaths_-_cirrhosis_and_other_chronic_liver_diseases',
       'Deaths_-_digestive_diseases', 'Deaths_-_hepatitis',
       'Deaths_-_neoplasms', 'Deaths_-_parkinson_disease',
       'Deaths_-_fire,_heat,_and_hot_substances', 'Deaths_-_malaria',
       'Deaths_-_drowning', 'Deaths_-_interpersonal_violence',
       'Deaths_-_hiv/aids', 'Deaths_-_drug_use_disorders',
       'Deaths_-_tuberculosis', 'Deaths_-_road_injuries',
       'Deaths_-_maternal_disorders', 'Deaths_-_neonatal_disorders',
       'Deaths_-_alcohol_use_disorders',
       'Deaths_-_exposure_to_forces_of_nature', 'Deaths_-_diar

In [15]:
df.columns = df.columns.str.replace("_-_", "_")

In [16]:
df.columns

Index(['Country', 'Year', 'Deaths_meningitis',
       'Deaths_lower_respiratory_infections',
       'Deaths_intestinal_infectious_diseases',
       'Deaths_protein-energy_malnutrition', 'Terrorism',
       'Deaths_cardiovascular_diseases',
       'Deaths_alzheimer_disease_and_other_dementias',
       'Deaths_chronic_kidney_disease', 'Deaths_chronic_respiratory_diseases',
       'Deaths_cirrhosis_and_other_chronic_liver_diseases',
       'Deaths_digestive_diseases', 'Deaths_hepatitis', 'Deaths_neoplasms',
       'Deaths_parkinson_disease', 'Deaths_fire,_heat,_and_hot_substances',
       'Deaths_malaria', 'Deaths_drowning', 'Deaths_interpersonal_violence',
       'Deaths_hiv/aids', 'Deaths_drug_use_disorders', 'Deaths_tuberculosis',
       'Deaths_road_injuries', 'Deaths_maternal_disorders',
       'Deaths_neonatal_disorders', 'Deaths_alcohol_use_disorders',
       'Deaths_exposure_to_forces_of_nature', 'Deaths_diarrheal_diseases',
       'Deaths_environmental_heat_and_cold_exposure',
  

In [17]:
df.rename(columns={"Deaths_hiv/aids" : "Deaths_aids"}, inplace = True)

In [18]:
df.columns = df.columns.str.replace("-","_")

In [19]:
df.rename(columns={"Life_expectancy_at_birth,_total_(years)" : "Total_life_expectancy_at_birth_in_years"}, inplace = True)

In [20]:
df.rename(columns={"Deaths_protein-energy_malnutrition" : "Deaths_protein_energy_malnutrition"}, inplace = True)

In [21]:
df.rename(columns={"Deaths_fire,_heat,_and_hot_substances" : "Deaths_by_fire_and_heat"}, inplace=True)

In [22]:
df.columns

Index(['Country', 'Year', 'Deaths_meningitis',
       'Deaths_lower_respiratory_infections',
       'Deaths_intestinal_infectious_diseases',
       'Deaths_protein_energy_malnutrition', 'Terrorism',
       'Deaths_cardiovascular_diseases',
       'Deaths_alzheimer_disease_and_other_dementias',
       'Deaths_chronic_kidney_disease', 'Deaths_chronic_respiratory_diseases',
       'Deaths_cirrhosis_and_other_chronic_liver_diseases',
       'Deaths_digestive_diseases', 'Deaths_hepatitis', 'Deaths_neoplasms',
       'Deaths_parkinson_disease', 'Deaths_by_fire_and_heat', 'Deaths_malaria',
       'Deaths_drowning', 'Deaths_interpersonal_violence', 'Deaths_aids',
       'Deaths_drug_use_disorders', 'Deaths_tuberculosis',
       'Deaths_road_injuries', 'Deaths_maternal_disorders',
       'Deaths_neonatal_disorders', 'Deaths_alcohol_use_disorders',
       'Deaths_exposure_to_forces_of_nature', 'Deaths_diarrheal_diseases',
       'Deaths_environmental_heat_and_cold_exposure',
       'Deaths_nutri

In [23]:
df.drop(columns="Unnamed: 0", inplace = True)

In [24]:
df.columns

Index(['Country', 'Year', 'Deaths_meningitis',
       'Deaths_lower_respiratory_infections',
       'Deaths_intestinal_infectious_diseases',
       'Deaths_protein_energy_malnutrition', 'Terrorism',
       'Deaths_cardiovascular_diseases',
       'Deaths_alzheimer_disease_and_other_dementias',
       'Deaths_chronic_kidney_disease', 'Deaths_chronic_respiratory_diseases',
       'Deaths_cirrhosis_and_other_chronic_liver_diseases',
       'Deaths_digestive_diseases', 'Deaths_hepatitis', 'Deaths_neoplasms',
       'Deaths_parkinson_disease', 'Deaths_by_fire_and_heat', 'Deaths_malaria',
       'Deaths_drowning', 'Deaths_interpersonal_violence', 'Deaths_aids',
       'Deaths_drug_use_disorders', 'Deaths_tuberculosis',
       'Deaths_road_injuries', 'Deaths_maternal_disorders',
       'Deaths_neonatal_disorders', 'Deaths_alcohol_use_disorders',
       'Deaths_exposure_to_forces_of_nature', 'Deaths_diarrheal_diseases',
       'Deaths_environmental_heat_and_cold_exposure',
       'Deaths_nutri

In [25]:
df[df.isna().any(axis=1)]

,Country,Year,Deaths_meningitis,Deaths_lower_respiratory_infections,Deaths_intestinal_infectious_diseases,Deaths_protein_energy_malnutrition,Terrorism,Deaths_cardiovascular_diseases,Deaths_alzheimer_disease_and_other_dementias,Deaths_chronic_kidney_disease,...,Life_satisfaction,Life_expectancy,Life_expectancy_at_birth,Share_of_population_below_poverty_line_2USD_per_day,Total_life_expectancy_at_birth_in_years,Total_population,Human_development_index,Mortality_rate_under_5_per_1000_live_births,Suicidy_mortality_rate_per_100000_population,Annual_CO2_emissions
0,Afghanistan,1990,6469.977091,22836.912346,295.382207,1607.703797,12.0,46498.085024,1959.215373,3155.356340,...,NaN,50.331,50.331,0.0,50.331,12412311.0,NaN,17.77,NaN,2.603238e+06
1,Afghanistan,1991,6347.158764,22325.633931,303.866598,1558.132316,68.0,46967.361037,1987.812878,3124.082631,...,NaN,50.999,50.999,0.0,50.999,13299016.0,NaN,17.17,NaN,2.427366e+06
2,Afghanistan,1992,6659.741428,23205.280750,317.750121,1617.721576,49.0,48355.558515,2025.102326,3192.278428,...,NaN,51.641,51.641,0.0,51.641,14485543.0,NaN,16.59,NaN,1.379462e+06
3,Afghanistan,1993,8068.386263,28229.715397,333.978331,1931.808553,0.0,50072.563899,2064.827903,3343.974014,...,NaN,52.256,52.256,0.0,52.256,15816601.0,NaN,16.04,NaN,1.333055e+06
4,Afghanistan,1994,9432.845127,32652.297630,348.419085,2351.629206,22.0,51416.815442,2101.020630,3480.299710,...,NaN,52.842,52.842,0.0,52.842,17075728.0,NaN,15.52,NaN,1.281602e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37786,World,1895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.485281e+09
37787,World,1896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.533711e+09
37788,World,1897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.606311e+09
37789,World,1898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.694277e+09


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37791 entries, 0 to 37790
Data columns (total 46 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Country                                                37791 non-null  object 
 1   Year                                                   37791 non-null  int64  
 2   Deaths_meningitis                                      6686 non-null   float64
 3   Deaths_lower_respiratory_infections                    6686 non-null   float64
 4   Deaths_intestinal_infectious_diseases                  6686 non-null   float64
 5   Deaths_protein_energy_malnutrition                     6686 non-null   float64
 6   Terrorism                                              6686 non-null   float64
 7   Deaths_cardiovascular_diseases                         6686 non-null   float64
 8   Deaths_alzheimer_disease_and_other_dementias  

In [27]:
df["Year"]
#df.dropna(inplace = True)

0        1990
1        1991
2        1992
3        1993
4        1994
         ... 
37786    1895
37787    1896
37788    1897
37789    1898
37790    1899
Name: Year, Length: 37791, dtype: int64

In [28]:
df.dtypes

Country                                                   object
Year                                                       int64
Deaths_meningitis                                        float64
Deaths_lower_respiratory_infections                      float64
Deaths_intestinal_infectious_diseases                    float64
Deaths_protein_energy_malnutrition                       float64
Terrorism                                                float64
Deaths_cardiovascular_diseases                           float64
Deaths_alzheimer_disease_and_other_dementias             float64
Deaths_chronic_kidney_disease                            float64
Deaths_chronic_respiratory_diseases                      float64
Deaths_cirrhosis_and_other_chronic_liver_diseases        float64
Deaths_digestive_diseases                                float64
Deaths_hepatitis                                         float64
Deaths_neoplasms                                         float64
Deaths_parkinson_disease 

In [31]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///deadline_database_nonans_geo.db", echo=True)
sqlite_connection = engine.connect()

sqlite_table = "Deadline_database"

df.to_sql(sqlite_table, sqlite_connection, if_exists="replace")

# close connection to DB
sqlite_connection.close()

In [32]:
from pycountry import countries

In [33]:
countries.lookup("Canada").alpha_3

'CAN'

In [36]:
countries

In [41]:
df["Country"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa',
       'Andean Latin America', 'Andorra', 'Angola', 'Antigua and Barbuda',
       'Argentina', 'Armenia', 'Australasia', 'Australasia & Oceania',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Caribbean', 'Central African Republic',
       'Central America & Caribbean', 'Central Asia', 'Central Europe',
       'Central Europe, Eastern Europe, and Central Asia',
       'Central Latin America', 'Central Sub-Saharan Africa', 'Chad',
       'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       'Czechoslovakia', 'Democratic Republic of Congo', 'Denmark',
 

In [52]:
# if any of the items/country is in list, then temporary dataframe has data, copy to new df
df2 = df[df["Country"].isin(countrylist)]

In [53]:
len(df2["Country"].unique())

159

In [54]:
countries

In [74]:
#df2["Country"] = df2["Country"].str.title()
#df2["Country"].unique()
df2["Country"] = df2["Country"].str.replace("Cote D'Ivoire", "Côte D'Ivoire")

/tmp/ipykernel_85493/3046792879.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Country"] = df2["Country"].str.replace("Cote D'Ivoire", "Côte D'Ivoire")


In [76]:
for c in df2["Country"]:
    df2["Code"] = countries.lookup(c).alpha_3    

/tmp/ipykernel_85493/2364609065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Code"] = countries.lookup(c).alpha_3


LookupError: Could not find a record for 'democratic republic of congo'

In [77]:
countries.search_fuzzy("Congo")

[Country(alpha_2='CG', alpha_3='COG', name='Congo', numeric='178', official_name='Republic of the Congo'),
 Country(alpha_2='CD', alpha_3='COD', name='Congo, The Democratic Republic of the', numeric='180')]

In [78]:
df2["Country"] = df2["Country"].str.replace("Democratic Republic Of Congo", "Congo, The Democratic Republic of the")

/tmp/ipykernel_85493/552642415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Country"] = df2["Country"].str.replace("Democratic Republic Of Congo", "Congo, The Democratic Republic of the")


In [82]:
df2["Country"] = df2["Country"].str.replace("Iran", "Iran, Islamic Republic of")

/tmp/ipykernel_85493/621326385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Country"] = df2["Country"].str.replace("Iran", "Iran, Islamic Republic of")


In [95]:
df2["Country"] = df2["Country"].str.replace("Laos", "Lao")

/tmp/ipykernel_85493/1747223804.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Country"] = df2["Country"].str.replace("Laos", "Lao")


In [96]:
for c in df2["Country"]:
    df2["Code"] = countries.lookup(c).alpha_3    

/tmp/ipykernel_85493/2364609065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Code"] = countries.lookup(c).alpha_3


LookupError: Could not find a record for 'russia'

In [99]:
countries.search_fuzzy("Russia")

[Country(alpha_2='RU', alpha_3='RUS', name='Russian Federation', numeric='643')]

In [100]:
df2["Country"] = df2["Country"].str.replace("Russia", "Russian Federation")

/tmp/ipykernel_85493/3469523331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Country"] = df2["Country"].str.replace("Russia", "Russian Federation")


In [101]:
for c in df2["Country"]:
    df2["Code"] = countries.lookup(c).alpha_3    

/tmp/ipykernel_85493/2364609065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Code"] = countries.lookup(c).alpha_3


LookupError: Could not find a record for 'south korea'

In [103]:
df2["Country"] = df2["Country"].str.replace("South Korea", "Korea, Republic of")

/tmp/ipykernel_85493/2265738018.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Country"] = df2["Country"].str.replace("South Korea", "Korea, Republic of")


In [104]:
for c in df2["Country"]:
    df2["Code"] = countries.lookup(c).alpha_3    

/tmp/ipykernel_85493/2364609065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Code"] = countries.lookup(c).alpha_3


LookupError: Could not find a record for 'syria'

In [105]:
countries.search_fuzzy("Syria")

[Country(alpha_2='SY', alpha_3='SYR', name='Syrian Arab Republic', numeric='760')]

In [106]:
df2["Country"] = df2["Country"].str.replace("Syria", "Syrian Arab Republic")

/tmp/ipykernel_85493/522305821.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Country"] = df2["Country"].str.replace("Syria", "Syrian Arab Republic")


In [113]:
for c in df2["Country"]:
    df2["Code2"] = countries.lookup(c).alpha_2    

/tmp/ipykernel_85493/803796691.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Code2"] = countries.lookup(c).alpha_2


In [115]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

In [119]:
for c in df2["Code2"]:
    df2["Continent"] = country_alpha2_to_continent_code(c)

/tmp/ipykernel_85493/3759123311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Continent"] = country_alpha2_to_continent_code(c)


In [120]:
df2.columns

Index(['Country', 'Year', 'Deaths_meningitis',
       'Deaths_lower_respiratory_infections',
       'Deaths_intestinal_infectious_diseases',
       'Deaths_protein_energy_malnutrition', 'Terrorism',
       'Deaths_cardiovascular_diseases',
       'Deaths_alzheimer_disease_and_other_dementias',
       'Deaths_chronic_kidney_disease', 'Deaths_chronic_respiratory_diseases',
       'Deaths_cirrhosis_and_other_chronic_liver_diseases',
       'Deaths_digestive_diseases', 'Deaths_hepatitis', 'Deaths_neoplasms',
       'Deaths_parkinson_disease', 'Deaths_by_fire_and_heat', 'Deaths_malaria',
       'Deaths_drowning', 'Deaths_interpersonal_violence', 'Deaths_aids',
       'Deaths_drug_use_disorders', 'Deaths_tuberculosis',
       'Deaths_road_injuries', 'Deaths_maternal_disorders',
       'Deaths_neonatal_disorders', 'Deaths_alcohol_use_disorders',
       'Deaths_exposure_to_forces_of_nature', 'Deaths_diarrheal_diseases',
       'Deaths_environmental_heat_and_cold_exposure',
       'Deaths_nutri

In [121]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///deadline_database_nonans_geo.db", echo=True)
sqlite_connection = engine.connect()

sqlite_table = "Deadline_database"

df.to_sql(sqlite_table, sqlite_connection, if_exists="replace")

# close connection to DB
sqlite_connection.close()

2021-12-06 20:15:45,385 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Deadline_database")
2021-12-06 20:15:45,385 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-06 20:15:45,387 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Deadline_database")
2021-12-06 20:15:45,387 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-06 20:15:45,389 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-12-06 20:15:45,390 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-06 20:15:45,391 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Deadline_database")
2021-12-06 20:15:45,392 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-06 20:15:45,394 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-12-06 20:15:45,395 INFO sqlalchemy.engine.Engine [raw sql] ('Deadline_database',)
2021-12-06 20:15:45,395 INFO sqlalchemy.engine.Engin